In [34]:
import pandas as pd
import numpy as np
import os
import sys 

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

original_file = "blindern_data_cleaning.csv"
data_path = os.path.join(os.getcwd(),"..", "data")
original_path = os.path.join(data_path, original_file)  
df = pd.read_csv(original_path, delimiter=";")


df['Tid'] = pd.to_datetime(df['Tid'], errors='coerce')


# calculate the average, median and standard deviation for each column
print(f"The average for each column is:\n{df.mean(numeric_only=True)}")
print(f"The median for each column is:\n{df.median(numeric_only=True)}")
print(f"The standard deviation for each column is:\n{df.std(numeric_only=True)}")

# calculate the seasonal average, median and standard deviation for each column
def season(dato):
    month=dato.month
    if month in [12,1,2]:
        return "Winter"
    elif month in [3,4,5]:
        return "Spring"
    elif month in [6,7,8]:
        return "Summer"
    elif month in [9,10,11]:
        return "Autumn"

df['season'] = df['Tid'].apply(season)



print(f"The average for each season is:\n{df.groupby('season').mean(numeric_only=True)}")
print(f"The median for each season is:\n{df.groupby('season').median(numeric_only=True)}")
print(f"The standard deviation for each season is:\n{df.groupby('season').std(numeric_only=True)}")





The average for each column is:
Makstemp         11.602115
Mintemp           4.153036
Middeltemp        7.617745
Snø               3.974116
Nedbør            2.423843
Middelvind        2.703982
Høye vindkast     5.564012
dtype: float64
The median for each column is:
Makstemp         11.4
Mintemp           3.6
Middeltemp        7.1
Snø               0.0
Nedbør            0.1
Middelvind        2.5
Høye vindkast     5.2
dtype: float64
The standard deviation for each column is:
Makstemp         9.050974
Mintemp          7.244604
Middeltemp       7.855064
Snø              9.927656
Nedbør           5.256624
Middelvind       1.134572
Høye vindkast    2.226089
dtype: float64
The average for each season is:
         Makstemp    Mintemp  Middeltemp        Snø    Nedbør  Middelvind  \
season                                                                      
Autumn  10.910390   4.863237    7.623676   0.124875  2.937463    2.691908   
Spring  11.807905   2.643577    6.929842   4.465415  1.684881